## 単語のトークン化

* 大文字小文字を区別しない
* !などの記号は無視してくれる
* カンマは認識しない

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 入力する文をlistで宣言
sentences = [
    'i love my dog',
    'I, love my cat'
    ]

# Tokenizerクラスの初期化
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
tokenizer = Tokenizer(num_words=100, oov_token="<OOV>")
# 引数
# num_words: 出現頻度の高い単語を何個まで使うか、これを超える単語は無視する
# oov_token: 未知語を表す文字列 (out of vocabulary)

# コーパスの各単語のインデックスを生成
tokenizer.fit_on_texts(sentences)

# コーパスの各単語のインデックス
word_index = tokenizer.word_index

# トークン列のリストを生成する
sequences = tokenizer.texts_to_sequences(sentences)

print("Word Index = ", word_index)
print("Sequences = ", sequences)
print("未出現の単語はindex1でprintされる", tokenizer.texts_to_sequences(["my hobby is playing with my dog."]))


Word Index =  {'<OOV>': 1, 'i': 2, 'love': 3, 'my': 4, 'dog': 5, 'cat': 6}
Sequences =  [[2, 3, 4, 5], [2, 3, 4, 6]]
未出現の単語はindex1でprintされる [[4, 1, 1, 1, 1, 4, 5]]


In [7]:
# 文中で出現頻度の高い単語の情報量は小さいので除去
def remove_stopwords(sentence):
    # 除去する単語のリスト
    stopwords = ["a", "and", "am", "i", "i'd", "you"]
    # 小文字に統一
    sentence = sentence.lower()

    words_list = sentence.split()  # 単語に分割
    words_list = [w for w in words_list if w not in stopwords]
    sentence = " ".join(words_list)  # スペースを挟んで結合
    return sentence

print(remove_stopwords("I am a student of Waseda."))

student of waseda.


## Padding

* pad_sequencesで単語列の長さが一定になるように0埋めする
* defaultだと0が先頭にくるが、padding='post'で末尾にすることができる
* maxlenは最大の長さを指定できる（指定しないと最大の長さになる）
* 単語列がmaxlenより長い場合は、defaultだと後ろからmaxlen分取得
* 単語列の先頭からmaxlen分取得したい場合は、truncating='post'を指定

In [5]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 配列の長さを一定にするためにPadding
padded = pad_sequences(sequences, maxlen=5, padding='post', truncating='post')
# https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences

print("Padded Sequences:\n", padded)

Padded Sequences:
 [[2 3 4 5 0]
 [2 3 4 6 0]]


## CSV read

In [ ]:
import csv

# GRADED FUNCTION: parse_data_from_file
def parse_data_from_file(filename):
    """
    Extracts sentences and labels from a CSV file

    Args:
        filename (string): path to the CSV file

    Returns:
        sentences, labels (list of string, list of string): tuple containing lists of sentences and labels
    """
    sentences = []
    labels = []
    with open(filename, 'r') as csvfile:
        ### START CODE HERE
        reader = csv.reader(csvfile, delimiter=",")
        header = next(reader)
        for row in reader:
            sentences.append(remove_stopwords(row[1]))
            labels.append(row[0])
        ### END CODE HERE
    return sentences, labels

## Train model

In [ ]:
import tensorflow as tf

vocab_size = 10000
max_length = 120
embedding_dim = 16

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup the training parameters
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

In [ ]:
num_epochs = 10
# model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

* 学習グラフを描画して、validation_lossが小さくなっていることを確認する
* 過学習が起きている場合、以下の対応が効果的かも
  * vocab_sizeを減らす
  * max_lengthを減らす

## LSTM

In [ ]:
import tensorflow as tf

# Hyperparameters
embedding_dim = 64
lstm_dim = 64
dense_dim = 64

# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Print the model summary
model.summary()

* single layerのLSTM
* multi layerのLSTM
* convolutional layer
* 